# Graph Analysis

## Description
Analyse:
- Topic detection metrics (Modularity, Ncut, NMI) across methods (BERTopic, Louvain, Leiden)
- Sentiment across topics ('politic', 'mental', 'health', 'emotion', 'tone', 'agency')

## Setup

### Libraries (Python)

In [1]:
import igraph as ig
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import sys
import copy
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Directories

In [2]:
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path().resolve()

SRC_DIR = BASE_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

DATA_DIR = BASE_DIR / 'data'
DATA_DIR.mkdir(exist_ok=True)
POSTS_DIR = DATA_DIR / 'posts'
POSTS_ALL_DIR = POSTS_DIR / 'all'
POSTS_FILTERED_DIR = POSTS_DIR / 'filtered'
POSTS_FILTERED_CLEAN_DIR = POSTS_FILTERED_DIR / 'clean'
COMMENTS_DIR = DATA_DIR / 'comments'
COMMENTS_CLEAN_DIR = COMMENTS_DIR / 'clean'
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_GRAPHS_DIR = RESULTS_DIR / 'graphs'
RESULTS_MODELS_DIR = RESULTS_DIR / 'models'
RESULTS_MODELS_DIR.mkdir(exist_ok=True)
RESULTS_PLOT_DIR = RESULTS_DIR / 'plots' 
RESULTS_MODELS_FILE = RESULTS_MODELS_DIR / 'model_results.csv'

### Libraries (Custom)

## Import Data

In [3]:
# Graph
filename = RESULTS_GRAPHS_DIR / 'topics_graph.graphml'
g_nx = nx.read_graphml(str(filename))
g = ig.Graph.from_networkx(g_nx)

print(g.summary())

IGRAPH U-W- 16 101 -- 
+ attr: edge_default (g), node_default (g), _nx_name (v), agency (v), emotion (v), health (v), leiden_topic (v), mental (v), node_strength (v), politic (v), tone (v), topic_label (v), weight (e)


In [4]:
# Graph df
filename_topic_df = DATA_DIR / 'topic_df.csv'
topic_df = pd.read_csv(filename_topic_df)
col_to_rename={'leiden_topic': 'Leiden Topic', 
               'politic': 'LIWC-Politic',
               'mental': 'LIWC-Mental',
               'health': 'LIWC-Health',
               'emotion': 'Emotion',
               'tone': 'Tone',
               'agency': 'Agency'}
topic_df.rename(columns=col_to_rename, inplace=True)
topic_df = topic_df[~topic_df['topic_label'].str.contains("Outliers", na=False)]

In [5]:
# Metrics df
metrics_df = pd.read_csv(RESULTS_MODELS_FILE)
metrics_df.rename(columns={'Unnamed: 0': 'Metric'}, inplace=True)

## Analyses

### Metrics

In [ ]:
# Plot results
models = metrics_df.columns[1:]
metrics = metrics_df['Metric'].tolist()

# Define a distinct color for each model
colors = {
    'BERTopic': '#d55a04',
    'BERTopic_reduced': '#e69f00',
    'Louvain': '#0071b2',
    'Leiden': '#56b3e9'
}

# Create subplot figure with 1 row, 3 columns
fig = make_subplots(rows=1, cols=3, subplot_titles=metrics)

for i, metric in enumerate(metrics, start=1):
    row = metrics_df[metrics_df['Metric'] == metric]
    
    for model in models:
        score = row[model].values[0]
        fig.add_trace(
            go.Bar(
                x=[model], 
                y=[score], 
                name=model,
                marker_color=colors[model],
                showlegend=(i == 1)  # show legend only on first subplot
            ),
            row=1, col=i
        )
        
    # Set y-axis range per subplot
    fig.update_yaxes(range=[0,1], row=1, col=i)

fig.update_layout(
    height=400,  # adjust height
    width=900,   # total width - will make each plot narrower
    title_text="Model Scores per Metric",
    barmode='group'
)

fig.show()

### Topic Sentiment

In [39]:
cols_to_plot = [col for col in topic_df.columns if col not in ['Leiden Topic', 'topic_label', 'node_strength', 'Tone']]

max_val = topic_df[cols_to_plot].max().max()
y_range = [-1.25, max(max_val, 2) * 1.1]

blue, vermillion = '#0071b2', '#d55a04'

for topic in topic_df['Leiden Topic']:
    data = topic_df.loc[topic_df['Leiden Topic'] == topic, cols_to_plot].iloc[0]
    df_plot = data.reset_index()
    df_plot.columns = ['feature', 'value']
    
    # Assign hex colors
    df_plot['color'] = df_plot['value'].apply(lambda x: blue if x >= 0 else vermillion)

    y_labels = [label + '  ' for label in cols_to_plot]

    df_plot['text_label'] = df_plot['value'].round(2).astype(str)

    fig = px.bar(
        df_plot, y='feature', x='value',
        labels={'value': 'Value'},
        range_x=y_range,
        color='color',
        color_discrete_sequence=[blue, vermillion],
        text='text_label'
    )

    fig.update_traces(orientation='h', showlegend=False, textposition='outside')

    """
    fig.update_yaxes(
        tickvals=cols_to_plot,
        ticktext=y_labels
    )
    """

    fig.update_yaxes(
        categoryorder='array',
        categoryarray=cols_to_plot,
        tickvals=cols_to_plot,
        ticktext=y_labels
    )
    
    fig.update_layout(
        xaxis=dict(
            dtick=0.5
        ),
        title=dict(
            text=f'{topic_df.loc[topic_df["Leiden Topic"] == topic, "topic_label"].values[0]}',
            x=0.5,
            xanchor='center'
        ),
        yaxis_title='',
        xaxis_title='',
        height=300,
        width=600,
        margin=dict(l=120, r=20, t=50, b=40)
    )

    filename = RESULTS_PLOT_DIR / 'sentiment' / f"topic_{topic}.png"
    fig.write_image(filename)
    fig.show()

